In [1]:
from __future__ import division, absolute_import

import sys
import random
import pickle

import pandas as pd
import numpy as np
import h5py
from plotnine import *
import seaborn as sns

#root
absPath = '/home/angela3/imbalance_pcm_benchmark/'
sys.path.insert(0, absPath)

from src.Target import Target
from src.imbalance_functions import *

np.random.seed(8)
random.seed(8)

Using TensorFlow backend.


In [2]:
protein_type = "GPCRs" #"kinases"
nfolds = 10

In [3]:
#Loading list of unique proteins 
with open("".join((absPath, "data/", protein_type, "/", protein_type, "_prots.pickle")), 'rb') as handle:
    unique_prots = pickle.load(handle)

In [4]:
strategies = ["resampling_before_clustering", "no_resampling", "resampling_after_clustering", "semi_resampling"]

ratios_df_completo = pd.read_csv("".join((absPath, "data/", protein_type, "/ratios_df_complete.csv")))

In [5]:
ratios_df_completo.drop(["Unnamed: 0", "Unnamed: 0.1"], axis=1, inplace=True)

In [6]:
ratios_df_completo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10772 entries, 0 to 10771
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   DeepAffinity Protein ID  10772 non-null  object 
 1   ratio_training           9902 non-null   float64
 2   ratio_test               10772 non-null  float64
 3   ratio_test_predicted     10772 non-null  float64
 4   acc                      10772 non-null  float64
 5   auroc                    8157 non-null   float64
 6   f1                       10772 non-null  float64
 7   balanced_acc             10772 non-null  float64
 8   mcc                      10772 non-null  float64
 9   strategy                 10772 non-null  object 
 10  fold                     10772 non-null  int64  
 11  Sequence                 10772 non-null  object 
 12  family                   10772 non-null  object 
 13  Uniprot ID               10772 non-null  object 
 14  n_interactions        

In [7]:
ratios_df_completo.head()

,DeepAffinity Protein ID,ratio_training,ratio_test,ratio_test_predicted,acc,auroc,f1,balanced_acc,mcc,strategy,fold,Sequence,family,Uniprot ID,n_interactions,len_seq
0,UYMO,0.477448,0.620536,0.575893,0.776786,0.859035,0.813433,0.774439,0.538935,resampling_before_clustering,0,MWNATPSEEPGFNLTLADLDWDASPGNDSLGDELLQLFPAPLLAGV...,GPCR,Q92847,1004,366
1,0BJE,0.487952,0.604651,0.627907,0.790698,0.841629,0.830189,0.776018,0.558389,resampling_before_clustering,0,MTPNSTGEVPSPIPKGALGLSLALASLIITANLLLALGIAWDRRLR...,GPCR,Q8TDU6,423,330
2,78TA,0.503030,0.500000,0.550000,0.950000,0.910000,0.952381,0.950000,0.904534,resampling_before_clustering,0,MMTPNSTELSAIPMGVLGLSLALASLIVIANLLLALGIALDRHLRS...,GPCR,Q80SS6,138,329
3,RJYX,0.435644,0.444444,0.777778,0.444444,0.350000,0.545455,0.475000,-0.059761,resampling_before_clustering,0,METNFSTPLNEYEEVSYESAGYTVLRILPLVVLGVTFVLGVLGNGL...,GPCR,P25090,112,351
4,95E2,0.525958,0.307190,0.385621,0.758170,0.856534,0.650943,0.751455,0.476653,resampling_before_clustering,0,MGPTSVPLVKAHRSSVSDYVNYDIIVRHYNYTGKLNISADKENSIK...,GPCR,P21453,2471,382


we should not use n_interactions because that the TOTAL NUMBER of interactions, but we are interested only in test!
we have to load te corresponding prediction on test file, check the shape and take this number as n_interactions

In [8]:
#TESTING

#strategy = "no_resampling"
#fold = 0
#pred_test_path = "".join((absPath, "data/", protein_type, "/", strategy, "/predictions/", str(fold), "/test.csv"))
#predtest_loaded = pd.read_csv(pred_test_path, index_col=False)
#predtest_loaded

In [9]:
#predtest_loaded.comp_ID.values.tolist()

In [10]:
list_rdm = []
for strategy in strategies:
    for fold in range(nfolds):
        for prot in unique_prots:
            dict_prot = {}
            rdm_comps = computing_random_baseline(ratios_df_completo, strategy, prot, fold, protein_type)
            dict_prot["prot"] = prot
            dict_prot["strategy"] = strategy
            dict_prot["random_baseline"] = rdm_comps 
            dict_prot["fold"] = fold
            #dict_prot["comps"] = comps
            dict_prot["idx"] = range(len(rdm_comps))
            list_rdm.append(dict_prot)

/home/angela3/imbalance_pcm_benchmark/src/imbalance_functions.py:228: RuntimeWarning: Mean of empty slice.
  ratio_training = subdf.loc[subdf["DeepAffinity Protein ID"]==prot, "ratio_training"].dropna().values.mean()
/home/angela3/venv/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [11]:
random_baseline_df = pd.DataFrame(list_rdm)

In [12]:
# list_rdm[5]

In [13]:
random_baseline_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19920 entries, 0 to 19919
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   prot             19920 non-null  object
 1   strategy         19920 non-null  object
 2   random_baseline  19920 non-null  object
 3   fold             19920 non-null  int64 
 4   idx              19920 non-null  object
dtypes: int64(1), object(4)
memory usage: 778.2+ KB


In [14]:
random_baseline_df.head()

,prot,strategy,random_baseline,fold,idx
0,UYMO,resampling_before_clustering,"[0.05300969096642462, 0.6334211492302029, 0.08...",0,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,0BJE,resampling_before_clustering,"[0.9273246008659501, 0.29542079486375, 0.86803...",0,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
2,78TA,resampling_before_clustering,"[0.34519733784771456, 0.16562915063271155, 0.6...",0,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
3,RJYX,resampling_before_clustering,"[0.579114280508877, 0.38983512814835286, 0.743...",0,"(0, 1, 2, 3, 4, 5, 6, 7, 8)"
4,95E2,resampling_before_clustering,"[0.07411693690391297, 0.021511819105442736, 0....",0,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."


In [15]:
list_cols = ["random_baseline", "idx"]
other_cols = list(set(random_baseline_df.columns) - set(list_cols))
exploded = [random_baseline_df[col].explode() for col in list_cols]
exploded_rdm_bsl = pd.DataFrame(dict(zip(list_cols, exploded)))
exploded_rdm_bsl = random_baseline_df[other_cols].merge(exploded_rdm_bsl, how="right", 
                                                        left_index=True, right_index=True)
exploded_rdm_bsl

,fold,prot,strategy,random_baseline,idx
0,0,UYMO,resampling_before_clustering,0.0530097,0
0,0,UYMO,resampling_before_clustering,0.633421,1
0,0,UYMO,resampling_before_clustering,0.087121,2
0,0,UYMO,resampling_before_clustering,0.416736,3
0,0,UYMO,resampling_before_clustering,0.854089,4
...,...,...,...,...,...
19917,9,MSAZ,semi_resampling,None,1
19918,9,9MDC,semi_resampling,None,0
19918,9,9MDC,semi_resampling,None,1
19919,9,EAPK,semi_resampling,None,0


In [16]:
exploded_rdm_bsl.head()

,fold,prot,strategy,random_baseline,idx
0,0,UYMO,resampling_before_clustering,0.0530097,0
0,0,UYMO,resampling_before_clustering,0.633421,1
0,0,UYMO,resampling_before_clustering,0.087121,2
0,0,UYMO,resampling_before_clustering,0.416736,3
0,0,UYMO,resampling_before_clustering,0.854089,4


In [17]:
exploded_rdm_bsl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1060851 entries, 0 to 19919
Data columns (total 5 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   fold             1060851 non-null  int64 
 1   prot             1060851 non-null  object
 2   strategy         1060851 non-null  object
 3   random_baseline  1040815 non-null  object
 4   idx              1060851 non-null  object
dtypes: int64(1), object(4)
memory usage: 48.6+ MB


In [18]:
exploded_rdm_bsl.to_csv("".join((absPath, "/data/", protein_type, "/random_baseline.csv")))

### Adding random_baseline to the ratios dataframe 

In [19]:
# Removing Nones
rdm_bsl_df = exploded_rdm_bsl.replace(to_replace='None', value=np.nan).dropna()

In [20]:
rdm_bsl_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1040815 entries, 0 to 19895
Data columns (total 5 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   fold             1040815 non-null  int64 
 1   prot             1040815 non-null  object
 2   strategy         1040815 non-null  object
 3   random_baseline  1040815 non-null  object
 4   idx              1040815 non-null  object
dtypes: int64(1), object(4)
memory usage: 47.6+ MB


In [21]:
rdm_bsl_df["label"] = rdm_bsl_df.apply(labelling, 1)
rdm_bsl_df.rename(axis=1, mapper={"prot":"DeepAffinity Protein ID"}, inplace=True)

In [22]:
ratios_per_fold = []
for strategy in strategies:
    for fold in range(nfolds):
        pred_test_path = "".join((absPath, "data/", protein_type, "/", strategy, "/predictions/", str(fold), "/test.csv"))
        test_data = pd.read_csv(pred_test_path, index_col=False)
        test_data["DeepAffinity Protein ID"] = test_data["DeepAffinity Protein ID"].str.replace("b\'", "")
        test_data["DeepAffinity Protein ID"] = test_data["DeepAffinity Protein ID"].str.replace("\'", "")
        
        for prot in unique_prots:
            len_subdf = test_data[test_data["DeepAffinity Protein ID"]==prot].shape[0]
            if len_subdf==0:
                continue
            test_data.loc[test_data["DeepAffinity Protein ID"]==prot, "idx"] = range(len_subdf)
        
        test_subdf = test_data[["DeepAffinity Protein ID", "y_test", "idx"]]
        
        subdf = rdm_bsl_df[(rdm_bsl_df.strategy == strategy) & (rdm_bsl_df.fold == fold)]
        ratios_pred_list, pred_test = predictions_ratios_list_bsl(strategy, subdf, unique_prots, fold)
        #merging both dataframes
        complete_rdm_subdf = pd.merge(subdf, test_subdf, how="left", on=["DeepAffinity Protein ID", "idx"])
        complete_rdm_subdf.dropna(how="any", axis=0, inplace=True)
        print(complete_rdm_subdf.info())
        
        list_metrics = []
        for prot in unique_prots:
            dict_prot = computing_metrics_per_prot_bsl(prot, complete_rdm_subdf)
            list_metrics.append(dict_prot)
        
        filt_list_metrics = []
        for val in list_metrics:
            if val != None:
                filt_list_metrics.append(val)
        
        metrics_df = pd.DataFrame(filt_list_metrics)
        ratios_df = pd.DataFrame(ratios_pred_list)
                
        #list_dictionaries.append(filt_list_metrics)
        #ratios_df = converting_ratios_to_df(training_ratios, test_ratios, pred_test_ratios, filt_list_metrics, strategy, prot_df)
        #ratios_df["fold"] = str(fold)
        #ratios_per_fold.append(ratios_df)
        metrics_df["fold"] = str(fold)
        metrics_df["strategy"] = strategy
        
        ratios_seq = pd.merge(ratios_df, metrics_df, "left", on=["DeepAffinity Protein ID", "fold", "strategy"])
        ratios_per_fold.append(ratios_seq)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36347 entries, 0 to 474
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   label                    36347 non-null  int64 
 1   DeepAffinity Protein ID  36347 non-null  object
dtypes: int64(1), object(1)
memory usage: 851.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 36347 entries, 0 to 36346
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   fold                     36347 non-null  int64 
 1   DeepAffinity Protein ID  36347 non-null  object
 2   strategy                 36347 non-null  object
 3   random_baseline          36347 non-null  object
 4   idx                      36347 non-null  object
 5   label                    36347 non-null  int64 
 6   y_test                   36347 non-null  int64 
dtypes: int64(3), object(4)
memory usage

/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/home/angela3/venv/lib

Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.


/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/angela3/venv/lib/

Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.


...

/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/home/angela3/venv/lib

Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.


/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/home/angela3/venv/lib

Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.


/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1465: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: 

In [23]:
rdm_baseline_df = pd.concat(ratios_per_fold)
rdm_baseline_df.dropna(axis=0, how="all", subset=["acc", "auroc", "f1", "balanced_acc", "mcc"], inplace=True)
rdm_baseline_df["fold"] = pd.to_numeric(rdm_baseline_df["fold"])

In [24]:
rdm_baseline_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9902 entries, 0 to 473
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   DeepAffinity Protein ID  9902 non-null   object 
 1   ratio_test_predicted     9902 non-null   float64
 2   strategy                 9902 non-null   object 
 3   fold                     9902 non-null   int64  
 4   acc                      9902 non-null   float64
 5   auroc                    7958 non-null   float64
 6   f1                       9902 non-null   float64
 7   balanced_acc             9902 non-null   float64
 8   mcc                      9902 non-null   float64
dtypes: float64(6), int64(1), object(2)
memory usage: 773.6+ KB


In [25]:
rdm_baseline_df = pd.merge(rdm_baseline_df, ratios_df_completo[["DeepAffinity Protein ID", 
                                                                "ratio_training", "strategy",
                                                               "fold", "ratio_test",
                                                               "Sequence", "Uniprot ID",
                                                               "family", "n_interactions", "len_seq"]],
                          how="left", on=["DeepAffinity Protein ID",  "strategy",
                                                               "fold"])

In [26]:
rdm_baseline_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9902 entries, 0 to 9901
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   DeepAffinity Protein ID  9902 non-null   object 
 1   ratio_test_predicted     9902 non-null   float64
 2   strategy                 9902 non-null   object 
 3   fold                     9902 non-null   int64  
 4   acc                      9902 non-null   float64
 5   auroc                    7958 non-null   float64
 6   f1                       9902 non-null   float64
 7   balanced_acc             9902 non-null   float64
 8   mcc                      9902 non-null   float64
 9   ratio_training           9902 non-null   float64
 10  ratio_test               9902 non-null   float64
 11  Sequence                 9902 non-null   object 
 12  Uniprot ID               9902 non-null   object 
 13  family                   9902 non-null   object 
 14  n_interactions          

In [27]:
rdm_baseline_df["is_baseline"] = True
ratios_df_completo["is_baseline"] = False

In [28]:
ratios_df_dict = pd.concat([ratios_df_completo, rdm_baseline_df])

In [29]:
ratios_df_dict.to_csv("".join((absPath, "data/", protein_type, "/ratios_df_rdm_baseline.csv")))